In [1]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sts
import seaborn as sns

%matplotlib inline
from bonding_curve_eq import *

## System Initialization

In [13]:
money_raised = 1000 # money raised in fiat, m
fiat_price = .1 # fiat per tokens, P_i
rho0 = .5 # reserve ratio

R0 = rho0*money_raised # initial reserve
S0 = money_raised/fiat_price # initial supply

kappa0 = 1/rho0 # curvature of bonding curve
alpha0 = 1 - rho0 # initial alpha
V0 = invariant_V0(R0, S0, kappa0)
P0 = spot_price(R0, V0, kappa0)

S_pos = 0 # supply tokens bonded to positive attestation, S_omega=1
S_neg = 0 # supply tokens bonded to negative attestation, S_omega=0

commitment = 800 # impact payers' payout commitment, C

## Agent Initialization

In [11]:
alpha_belief = alpha0 # agent's private belief of alpha, omega
price_belief = P0 # agent's private belief of price, p_i

attestations_pos = 0 # positive attestations held by agent, Q_omega=1
attestations_neg = 0 # negative attestations held by agent, Q_omega=0

#param for open loop
rules = ["martin", "step","ramp", "sin"]

## Parameters

In [12]:
params = {
    'kappa': [kappa0],
    'alpha': [alpha0],
    'invariant_V0': [V0],
    'invariant_I0': [I0],
    'rule' : rules,
    'dP' : ['N/A', P0/4, P0/1000, P0/2],
    'period': ['N/A', 2000,2000,2000]}

## Initial Conditions

In [14]:
initial_conditions = {'reserve': R0,
                      'supply': S0,
                      'price': P0,
                      'spot_price': P0,
                      'alpha': alpha0,
                      'spot_alpha': alpha0,
                      'rho': rho0
                      }

In [2]:
def set_action(params, step, sL, s):
    
    R = s['reserve']
    S = s['supply']
    V0 = params['invariant_V0']
    P = s['spot_price']
    alpha = s['spot_alpha']
    period = params['period']
    
    if params['rule'] == 'sin':
        new_price = P0 + params['dP']*np.sin(2*np.pi*s['timestep']/period)
        print("new_price = ", new_price)
        
    if P > price_belief_a:
        mech = 'burn'
    elif P < price_belief_a:
        mech = 'bond'
    else: 
        # don't trade 
        print("No trade")
        
    
    if alpha > alpha_belief_a:
        mech = 'attest_neg'
    elif alpha < alpha_belief_a:
        mech = 'attest_pos'
    else:
        # don't attest
        print("No attestation")
        
    ####### Any syntax related changes required here? #######
    # design pattern for action
    
    action = {
                'mech':mech, 
                'amt_b':amt_b,
                'amt_a':amt_a,
                'p_in':new_price,
                'price_belief':price_belief(amt_b),
                'alpha_in': new_alpha,
                'alpha_belief': alpha_belief(amt_a),
                'posterior':{}
    }
    
    if action['mech'] == 'bond':

        dS, price_belief = mint(amt_b, R, S, V0, params['kappa'])
        R = R + amt_b
        S = S + dS
        P = spot_price(R, V0, kappa)

    elif action['mech'] == 'burn':
        dR, price_belief = withdraw(amt_b, R, S, V0, params['kappa'])
        R = R - dR
        S = S - amt_b
        P = spot_price(R, V0, kappa)
        
    if action['mech'] == 'attest_pos':
        
        dQ1, alpha_belief = attest_pos(amt_a, S1, C, I0, params['kappa'])
        S1 = S1 + amt_a
        Q1 = Q1 + dQ1
        alpha = spot_alpha(S, I0, kappa, C)
    
    elif action['mech'] == 'attest_neg':
        
        dQ0, alpha_belief = attest_neg(amt_a, S0, C, I0, params['kappa'])
        S0 = S0 + amt_a
        Q0 = Q0 + dQ0
        alpha = spot_alpha(S, I0, kappa, C)
    
    
    action['posterior'] = {'S':S, 'R':R,'P':P, 'S1':S1, 'Q1':Q1, 'S2':S2, 'Q2':Q2, 'alpha':alpha}
    
    return {'action': action}

In [ ]:
def set_action(params, step, sL, s):
    
    R = s['reserve']
    S = s['supply']
    V = params['invariant_V']
    I = params['invariant_I']
    P = s['spot_price']
    alpha = s['spot_alpha']
    kappa = params['kappa']
    period = params['period']
    
    if params['rule'] == 'sin':
        new_price = P0 + params['dP']*np.sin(2*np.pi*s['timestep']/period)
        print("new_price = ", new_price)
        
    if P > price_belief_a:
        mech = 'burn'
    elif P < price_belief_a:
        mech = 'bond'
    else: 
        # don't trade 
        print("No trade")
        
    
    if alpha > alpha_belief_a:
        mech = 'attest_neg'
    elif alpha < alpha_belief_a:
        mech = 'attest_pos'
    else:
        # don't attest
        print("No attestation")
    
    action = {
                'mech':mech, 
                'amt_b':amt_b,
                'amt_a':amt_a,
                'p_in':new_price,
                'price_belief':price_belief(amt_b),
                'alpha_in': new_alpha,
                'alpha_belief': alpha_belief(amt_a),
                'posterior':{}
    }
    
    if action['mech'] == 'bond':
        dS, price_belief = bond(amt_b, R, S, V, params['kappa'])
        R = R + amt_b
        S = S + dS
        I = R + (C*alpha)
        kappa = kappa(deltaR, R, S, V, I, alpha)
        P = spot_price(R, V, kappa)
        

    elif action['mech'] == 'burn':
        dR, price_belief = withdraw(amt_b, R, S, V, params['kappa'])
        R = R - dR
        S = S - amt_b
        I = R + (C*alpha)
        kappa = kappa(dR, R, S, V, I, alpha)
        P = spot_price(R, V, kappa)
        
    if action['mech'] == 'attest_pos':      
        dQ1, alpha_belief = attest_pos(amt_a, S1, C, I, params['kappa'])
        S1 = S1 + amt_a
        Q1 = Q1 + dQ1
        alpha = spot_alpha(S, I, kappa, C)
        kappa = kappa(dR, R, S, V, I, alpha)
        P = spot_price()
        V = invariant_V(R, S, kappa)
    
    elif action['mech'] == 'attest_neg':       
        dQ0, alpha_belief = attest_neg(amt_a, S0, C, I, params['kappa'])
        S0 = S0 + amt_a
        Q0 = Q0 + dQ0
        alpha = spot_alpha(S, I, kappa, C)
        kappa = kappa(dR, R, S, V, I, alpha)
        P = spot_price()
        V = invariant_V(R, S, kappa)
    
    action['posterior'] = {'S':S, 'R':R, 'P':P, 'S1':S0, 'S1':S1, 'Q0':Q0, 'Q1':Q1, 'kappa':kappa, 'alpha':alpha, 'I':I, 'V':V}
    
    return {'action': action}

In [ ]:
def update_S(params, step, sL, s, _input):
    
    action = _input['action']
    S = action['posterior']['S']
    
    key = 'supply'
    value = S
    
    return (key, value)

def update_R(params, step, sL, s, _input):
    
    action = _input['action']
    R = action['posterior']['R']
    
    key = 'reserve'
    value = R
    
    return (key, value)

def update_P(params, step, sL, s, _input):
    
    action = _input['action']
    P = action['posterior']['P']
    
    key = 'spot_price'
    value = P
    
    return (key, value)

def update_price_belief(params, step, sL, s, _input):
    
    action = _input['action']
    price_belief = action['price_belief']
    
    key = 'output_price'
    value = price_belief
    
    return (key, value)

def capture_P_in(params, step, sL, s, _input):
    
    P_in = _input['action']['p_in']
    if P_in == 0:
        print(_input['action']) 
    
    key = 'price'
    value = P_in
    
    return (key, value)

def update_alpha_belief(params, step, sL, s, _input):
    
    action = input['action']
    alpha_belief = action['alpha_belief']
    
    key = 'output_alpha'
    value = alpha_belief
    
    return (key, value)

def capture_alpha_in(params, step, sL, s, _input):
    
    alpha_in = _input['action']['alpha_in']
    if alpha_in == 0:
        print(_input['action']) 
    
    key = 'alpha'
    value = alpha_in
    
    return (key, value)